In [2]:
import pandas as pd
import numpy as np

In [3]:
df_prices = pd.read_csv("../datos/crudos/car_prices.csv")
df_fuel = pd.read_csv("../datos/crudos/Fuel_Consumption_2000-2022.csv")

In [4]:
# columnas
df_prices.columns = df_prices.columns.str.lower().str.strip()
df_fuel.columns = df_fuel.columns.str.lower().str.strip()

# tipos
for col in ["year", "sellingprice", "mmr", "odometer"]:
    if col in df_prices.columns:
        df_prices[col] = pd.to_numeric(df_prices[col], errors="coerce")
df_fuel["year"] = pd.to_numeric(df_fuel["year"], errors="coerce")

# texto
df_prices["make"] = df_prices["make"].astype(str).str.upper().str.strip()
df_prices["model"] = df_prices["model"].astype(str).str.upper().str.strip()

df_fuel["make"] = df_fuel["make"].astype(str).str.upper().str.strip()
df_fuel["model"] = df_fuel["model"].astype(str).str.upper().str.strip()

In [5]:
df_prices_join = df_prices.dropna(subset=["year", "make", "model"]).copy()
df_fuel_join = df_fuel.dropna(subset=["year", "make", "model"]).copy()

In [6]:
df_merged_check = df_prices_join.merge(
    df_fuel_join,
    on=["year", "make", "model"],
    how="left",
    indicator=True
)

In [7]:
df_merged_check["_merge"].value_counts()

_merge
both          834500
left_only     208101
right_only         0
Name: count, dtype: int64

In [8]:
df_final = df_merged_check.drop(columns=["_merge"])

In [9]:
df_final.shape

(1042601, 26)

In [10]:
df_final.to_csv("../datos/procesados/dataset_final_coches.csv", index=False)

In [11]:
import os
os.path.exists("../datos/procesados/dataset_final_coches.csv")

True

In [12]:
df_final.shape
df_final.columns

Index(['year', 'make', 'model', 'trim', 'body', 'transmission_x', 'vin',
       'state', 'condition', 'odometer', 'color', 'interior', 'seller', 'mmr',
       'sellingprice', 'saledate', 'vehicle class', 'engine size', 'cylinders',
       'transmission_y', 'fuel', 'fuel consumption', 'hwy (l/100 km)',
       'comb (l/100 km)', 'comb (mpg)', 'emissions'],
      dtype='object')

In [13]:
df_final = df_final.drop(columns=["transmission_y"])
df_final = df_final.rename(columns={"transmission_x": "transmission"})

In [14]:
df_final.columns

Index(['year', 'make', 'model', 'trim', 'body', 'transmission', 'vin', 'state',
       'condition', 'odometer', 'color', 'interior', 'seller', 'mmr',
       'sellingprice', 'saledate', 'vehicle class', 'engine size', 'cylinders',
       'fuel', 'fuel consumption', 'hwy (l/100 km)', 'comb (l/100 km)',
       'comb (mpg)', 'emissions'],
      dtype='object')

In [15]:
df_final.dtypes

year                  int64
make                 object
model                object
trim                 object
body                 object
transmission         object
vin                  object
state                object
condition           float64
odometer            float64
color                object
interior             object
seller               object
mmr                 float64
sellingprice        float64
saledate             object
vehicle class        object
engine size         float64
cylinders           float64
fuel                 object
fuel consumption    float64
hwy (l/100 km)      float64
comb (l/100 km)     float64
comb (mpg)          float64
emissions           float64
dtype: object

In [16]:
cols_numeric = [
    "sellingprice",
    "mmr",
    "odometer",
    "condition",
    "engine size",
    "cylinders",
    "fuel consumption",
    "hwy (l/100 km)",
    "comb (l/100 km)",
    "comb (mpg)",
    "emissions"
]

In [17]:
for col in cols_numeric:
    df_final[col] = (
        df_final[col]
        .astype(str)
        .str.replace(",", ".", regex=False)
        .astype(float)
    )

In [18]:
df_final[cols_numeric].dtypes

sellingprice        float64
mmr                 float64
odometer            float64
condition           float64
engine size         float64
cylinders           float64
fuel consumption    float64
hwy (l/100 km)      float64
comb (l/100 km)     float64
comb (mpg)          float64
emissions           float64
dtype: object

In [ ]:
df_final[cols_numeric].isna().sum()

sellingprice            26
mmr                    130
odometer               140
condition            20809
engine size         208101
cylinders           208101
fuel consumption    208101
hwy (l/100 km)      208101
comb (l/100 km)     208101
comb (mpg)          208101
emissions           208101
dtype: int64

In [20]:
df_final = df_final.dropna(subset=["sellingprice", "year"])

In [21]:
df_consumo = df_final.dropna(subset=["fuel consumption"])

In [25]:
# Filtrar por rango temporal coherente
df_final = df_final[
    (df_final["year"] >= 2000) &
    (df_final["year"] <= 2022)
]

In [26]:
df_final.shape
df_final["year"].min(), df_final["year"].max()

(np.int64(2000), np.int64(2015))

In [27]:
df_final.shape

(1033042, 25)

In [34]:
df_final.to_csv(
    "../datos/procesados/dataset_final_coches.csv",
    index=False,
    sep=",",
    decimal="."
)

In [35]:
import os
os.path.exists("../datos/procesados/dataset_final_coches.csv")

True